# **Typical architecture of a classification neural network**
분류 신경망의 일반적인 구조

분류 신경망의 구조는 당신이 연구하고 있는 문제에 따라 매우 다양함

* Input layer
* Hidden layers
* Oneput

나머지는 대부분 데이터 분석가가 모델을 만드는 데 달려 있습니다.

다음은 분류 신경망에서 자주 사용하는 몇 가지 표준 값입니다.


| **Hyperparameter** | **Binary Classification** | **Multiclass classification** |
| --- | --- | --- |
| Input layer shape | Same as number of features (e.g. 5 for age, sex, height, weight, smoking status in heart disease prediction) | Same as binary classification |
| Hidden layer(s) | Problem specific, minimum = 1, maximum = unlimited | Same as binary classification |
| Neurons per hidden layer | Problem specific, generally 10 to 100 | Same as binary classification |
| Output layer shape | 1 (one class or the other) | 1 per class (e.g. 3 for food, person or dog photo) |
| Hidden activation | Usually [ReLU](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning) (rectified linear unit) | Same as binary classification |
| Output activation | [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) | [Softmax](https://en.wikipedia.org/wiki/Softmax_function) |
| Loss function | [Cross entropy](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression) ([`tf.keras.losses.BinaryCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) in TensorFlow) | Cross entropy ([`tf.keras.losses.CategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) in TensorFlow) |
| Optimizer | [SGD](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) (stochastic gradient descent), [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) | Same as binary classification |


# **Prerequisite Python Modules**

먼저 일부 소프트웨어를 Python 환경에 로드

In [ ]:
import tensorflow as tf
import numpy as np                   # advanced math library
import matplotlib.pyplot as plt      # MATLAB like plotting routines
import random                        # for generating random numbers

from tensorflow.keras.models import Sequential  # Model type to be used
from tensorflow.keras.layers import Dense, InputLayer, Flatten, Dropout, Activation
from tensorflow.keras.datasets import mnist     # MNIST dataset is included in Keras
from tensorflow.keras.utils import to_categorical, plot_model

print(tf.__version__) # find the version number (should be 2.x+)

# 그래픽카드 유무 확인 및 메모리 확장 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print('사용가능한 GPU 갯수: ',len(gpus), '\n')
      
  try:
    # 프로그램이 실행되어 더 많은 GPU 메모리가 필요하면, 텐서플로 프로세스에 할당된 GPU 메모리 
    # 영역을 확장할 수있도록 허용
    tf.config.experimental.set_memory_growth(gpus[0], True)

  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

# 설치된 GPU 상세내용 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# **Loading Training Data**

MNIST 데이터셋은 Keras 내에서 편리하게 번들로 제공되며 Python에서 일부 기능을 쉽게 분석할 수 있습니다.

In [ ]:
# The MNIST data is split between 60,000 28 x 28 pixel training images and 10,000 28 x 28 pixel images
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

In [ ]:
X_train = X_train.reshape(60000, 784) # reshape 60,000 28 x 28 matrices into 60,000 784-length vectors.
X_test = X_test.reshape(10000, 784)   # reshape 10,000 28 x 28 matrices into 10,000 784-length vectors.

X_train = X_train.astype('float32')   # change integers to 32-bit floating point numbers
X_test = X_test.astype('float32')
print
X_train /= 255                        # normalize each value for each pixel for the entire vector for each input
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

클래스(고유 숫자)를 원핫 형식으로 수정

```
0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
etc.
```

만약 우리 네트워크의 최종 산출물이 이 클래스들 중 하나에 매우 가깝다면, 아마도 다음 예와 같음:

```
[0, 0.94, 0, 0, 0, 0, 0.06, 0, 0, 0]
```
그러면 이미지가 숫자 1의 이미지일 가능성이 가장 높습니다.

In [ ]:
X_test[0]
nb_classes = 10 # number of unique digits

Y_train = to_categorical(y_train,nb_classes)
Y_test = to_categorical(y_test,nb_classes)
print(X_test[0].shape)

# **Building a 3-layer fully connected network (FCN)**
3계층 완전 연결 네트워크(FCN) 구축

<img src="https://github.com/AviatorMoser/keras-mnist-tutorial/blob/master/figure.png?raw=1" />

# **DNN Model**

In [ ]:
# 순차 모델은 레이어의 선형 스택이며 매우 일반적입니다.

model = Sequential()

# 첫 번째 숨겨진 층은 512개의 노드 세트입니다(인공 뉴런).
# 각 노드는 각 입력 벡터로부터 요소를 수신하고 일부 가중치와 편향을 적용합니다.
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # Rerectified Linear Unit(ReLU)는 모든 음의 입력을 0으로 변환
                              # 노드의 양수값은 변경되지 않음

model.add(Dropout(0.2)) # 과적합을 방지하기 위해 20%노드를 비활성화

# 두 번째 숨겨진 레이어는 첫 번째 레이어와 동일하게 나타납니다.
# 그러나, 각각의 512 노드가 입력 이미지 데이터로부터 784개의 입력을 수신하는 대신,
# 첫 번째 512 노드 레이어의 출력에서 512개의 입력을 수신합니다.
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# 10개의 뉴런으로 구성된 최종 레이어는 이전 512-노드 레이어에 완전히 연결됩니다.
# FCN의 최종 레이어는 원하는 클래스 수와 같아야 합니다(이 경우 10).
model.add(Dense(10))
model.add(Activation('softmax'))
# "softmax" 활성함수는 가능한 n가지 결과에 대한 확률 분포를 나타냅니다.
# 값은 모두 음수가 아니며 합은 1입니다.

model.summary()

# **Compiling the model**

* Keras는 Theano와 TensorFlow 위에 구축되어 있음. 두 패키지 모두 Python에서 계산 그래프를 정의할 수 있음<br> 
* 그러면 Python 인터프리터의 오버헤드 없이 CPU 또는 GPU에서 효율적으로 컴파일되고 실행됩니다.<br>

* 모형을 컴파일할 때 Keras는 손실 함수와 최적기를 지정하도록 요청합니다.<br> 
* 여기서 사용할 손실 함수를 범주형 교차 엔트로피라고 하며, 두 확률 분포를 비교하는 데 적합한 손실 함수입니다.<br>

* 우리의 예측은 10개의 다른 자릿수에 대한 확률 분포(예: "이 이미지가 3, 10% 확실합니다. 8, 5% 확실합니다."). <br>
* 목표는 정확한 범주의 확률 분포이고 그 밖의 모든 범주의 확률 분포는 0입니다.
* 교차 엔트로피는 예측 분포가 목표 분포와 얼마나 다른지를 나타내는 측도입니다.<br>

* 옵티마이저는 모델이 경사하강을 통해 얼마나 빨리 학습하는지 결정하는 데 도움이 됩니다. 
* 내려가는 속도를 학습 속도라고 합니다.<br>

<img src = "https://randlow.github.io/images/ml/gradient-descent.png" >

<img src = "https://srdas.github.io/DLBook/DL_images/TNN2.png" >

그러면 학습률이 작을수록 더 나을까요? 그렇지 않아!<br> 옵티마이저는 손실 함수의 글로벌 최소값을 무시한 채 로컬 최소값에 고착되지 않는 것이 중요합니다.<br> 때때로 그것은 지역 최소치에서 벗어나기 위해 더 큰 학습률을 시도한다는 것을 의미합니다.

<img src = 'https://miro.medium.com/max/2060/1*9DVEXY4X0eNAx_ZHoD_PPA.png' >

In [ ]:
# Adam Optimizer를 학습에 사용
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# **Train the model**

배치 크기는 손실 함수, 기울기 및 후방 전파를 계산하는 데 사용되는 단계당 데이터 양을 결정합니다. <br>배치 크기가 크면 네트워크에서 교육을 더 빨리 완료할 수 있습니다. 그러나 교육 속도 이외에 고려해야 할 다른 요소도 있습니다.

배치 크기가 너무 크면 손실 함수의 로컬 최소값이 평활화되어 글로벌 최소값을 찾은 것으로 생각되어 최적화 프로그램이 한 개로 안착됨

배치 크기가 너무 작으면 노이즈가 매우 많은 손실 함수가 생성되고 최적화 프로그램이 전역 최소값을 찾지 못할 수 있음

따라서 배치 크기가 적절하면 시행착오를 겪어야 찾을 수 있습니다.

In [ ]:
import time
start = time.time()
history = model.fit(X_train, Y_train, batch_size = 128, epochs = 5,
                    validation_data = (X_test, Y_test), verbose = 1)
end = time.time()
print('Execution time in seconds = ', end-start)

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(title="Loss and Accuracy")

순서대로 두 숫자는 교육 세트의 네트워크 손실 함수 값과 교육 데이터에 대한 네트워크의 전반적인 정확도를 나타냅니다.<br> 하지만 훈련하지 않은 데이터는 어떻게 되나요?

# **Check the accuracy for test data.**

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Test score:', loss)
print('Test accuracy:', accuracy)

# **Evaluate Model's Accuracy on Test Data**
테스트 데이터에 대한 모형의 정확도 평가

# **분류성능평가지표 - 혼동행렬(Confusion Matrix)**
모델을 평가하는 요소는 결국, 모델이 내놓은 답과 실제 정답의 관계로써 정의를 내릴 수 있습니다.<br> 정답이 True와 False로 나누어져있고, 분류 모델 또한 True False의 답을 내놓습니다.<br> 그렇게 하면, 이진분류(Binary Classification)의 경우 아래와 같이 2x2 matrix로 case를 나누어볼 수 있습니다.

img src="https://t1.daumcdn.net/cfile/tistory/99DC064C5BE056CE10">

위와 같이 구분한 표를 혼동행렬(Confusion Matrix)이라고 합니다
```
  - True Positive(TP) : 실제 True인 정답을 True라고 예측 (정답)
  - False Positive(FP) : 실제 False인 정답을 True라고 예측 (오답)
  - False Negative(FN) : 실제 True인 정답을 False라고 예측 (오답)
  - True Negative(TN) : 실제 False인 정답을 False라고 예측 (정답)
```
    - 거짓 긍정(FP): 우리는 그렇다고 예측했지만, 실제로는 질병 가지지않음
    - 거짓 부정(FN): 우리는 아니라고 예상했지만, 실제로는 질병을 가짐
    - 참 긍정(TP): 우리는 그렇다고 예측했고, 실제로도 질병을 가짐
    - 참 부정(TN): 우리는 아니라고 예상했고, 실제로도 질병을 가지지 않음

 <hr>

# **분류성능평가지표 - Precision(정밀도)과 Recall(재현율)**

  - Precision(정밀도): 모델이 True라고 분류한 것 중에서 실제 True인 것의 비율

    $$Precision =  \frac {TP}{TP+FP}$$

  - Recall(재현율): 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율

    $$Recall =  \frac {TP}{TP+FN}$$

    통계학에서는 sensitivity로, 그리고 다른 분야에서는 hit rate라는 용어로도 사용합니다. 실제 날씨가 맑은 날 중에서 모델이 맑다고 예측한 비율을 나타낸 지표인데, 정밀도(Precision)와 True Positive의 경우를 다르게 바라보는 것입니다. 즉, Precision이나 Recall은 모두 실제 True인 정답을 모델이 True라고 예측한 경우에 관심이 있으나, 바라보고자 하는 관점만 다릅니다. Precision은 모델의 입장에서, 그리고 Recall은 실제 정답(data)의 입장에서 정답을 정답이라고 맞춘 경우를 바라보고 있습니다. 

      <img src= "https://t1.daumcdn.net/cfile/tistory/999D9C465BE116E43C">

  A는 실제 날씨가 맑은 날입니다. 그리고 B는 모델에서 날씨가 맑은 날이라고 예측한 것입니다.
  $$Precison =\frac{b}{b+c},  \hspace{2ex}  Recall = \frac{b}{a+b}$$

  모델의 입장에서 모두 맑은 날이라고만 예측하는 경우를 생각해봅시다. 그렇게 되면 TN(d)의 영역이 줄어들게 되고 그에 따라 FN(a)의 영역 또한 줄게 됩니다. 그러므로 Recall은 분모의 일부인 FN(a)영역이 줄기 때문에 Recall은 100%가 됩니다. 

    <img src = "https://t1.daumcdn.net/cfile/tistory/9951F44B5BE1205F1E">

  위 그림의 왼쪽 Case에서 Recall은 20 / 50 = 40%, Precision = 20 / 60 = 33.3% 입니다. 그리고 분류모델이 모두 True라고 예측한 오른쪽의 case에서의 recall은 FN = 0이므로 100%이지만 그에 따라 FP가 늘어서 precision은 20/100 = 20%가 되었습니다. 이처럼 precision과 recall은 모두 높은 것이 좋지만, trade-off 관계에 있어서 함께 늘리기가 힘듭니다.

 Precision과 Recall은 trade-off관계이며 상호보완적으로 해석해야하며, 두 지표가 모두 높을 수록 좋은 모델입니다.




 <hr>

## **분류성능평가지표 - Accuracy(정확도) 와 F1 Score**

  - Accuracy(정확도)

  $$Accuracy(정확도) = \frac {TP + TN}{TP+FN+FP+FN}$$

  앞서 설명한 두 지표는 모두 True를 True라고 옳게 예측한 경우에 대해서만 다루었습니다. 하지만, False를 False라고 예측한 경우도 옳은 경우입니다. 이때, 해당 경우를 고려하는 지표가 바로 정확도(Accuracy)입니다. 

  정확도는 **가장 직관적으로 모델의 성능을 나타낼 수 있는 평가 지표**입니다. 
  
  하지만, 여기서 고려해야하는 것이 있습니다. 바로 domain의 편중(bias)입니다. 만약 우리가 예측하고자 하는 한달 동안이 특정 기후에 부합하여 비오는 날이 흔치 않다고 생각해보죠. 이 경우에는 해당 data의 domain이 불균형하게되므로 맑은 것을 예측하는 성능은 높지만, 비가 오는 것을 예측하는 성능은 매우 낮을 수 밖에 없습니다. 따라서 이를 보완할 지표가 필요합니다.

  - F1 Score: Precision과 Recall의 [조화평균(역수의 산술 평균의 역수)](https://ko.wikipedia.org/wiki/%EC%A1%B0%ED%99%94_%ED%8F%89%EA%B7%A0)입니다. 

 $$F1  = 2 \times \frac {1}{\frac{1}{Precision} + \frac{1}{recall}}  = 2 \times \frac{Precision \times Recall}{Precision+Recall}$$ 





| **Metric name/Evaluation method** | **Defintion** | **Code** |
| --- | --- | --- |
| Accuracy | Out of 100 predictions, how many does your model get correct? E.g. 95% accuracy means it gets 95/100 predictions correct. | [`sklearn.metrics.accuracy_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) or [`tf.keras.metrics.Accuracy()`](tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy) |
| Precision | Proportion of true positives over total number of samples. Higher precision leads to less false positives (model predicts 1 when it should've been 0). | [`sklearn.metrics.precision_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html) or [`tf.keras.metrics.Precision()`](tensorflow.org/api_docs/python/tf/keras/metrics/Precision) |
| Recall | Proportion of true positives over total number of true positives and false negatives (model predicts 0 when it should've been 1). Higher recall leads to less false negatives. | [`sklearn.metrics.recall_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) or [`tf.keras.metrics.Recall()`](tensorflow.org/api_docs/python/tf/keras/metrics/Recall) |
| F1-score | Combines precision and recall into one metric. 1 is best, 0 is worst. | [`sklearn.metrics.f1_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) |
| [Confusion matrix](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/)  | Compares the predicted values with the true values in a tabular way, if 100% correct, all values in the matrix will be top left to bottom right (diagnol line). | Custom function or [`sklearn.metrics.plot_confusion_matrix()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) |



컴파일할 때 측정 지표 매개변수에 ""정확성"을 전달했으므로, 이 매개변수에 대해 평가()를 호출하면 손실과 정확성뿐만 아니라 손실도 반환

In [ ]:
# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
# and Made with ML's introductory notebook - https://github.com/madewithml/basics/blob/master/notebooks/09_Multilayer_Perceptrons/09_TF_Multilayer_Perceptrons.ipynb
import itertools
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
             horizontalalignment="center",
             color="white" if cm[i, j] > threshold else "black",
             size=text_size)

In [ ]:
# Get predicted class for X_test by converting all of the predictions from probabilities to labels
# 모든 예측을 확률에서 레이블로 변환하여 X_test의 예측 클래스를 가져옵니다.
y_probs = model.predict(X_test)
#print(tf.argmax(pred[0:10]))
print(y_probs[0:10])

y_pred=y_probs.argmax(axis=1)
print(y_pred[0:10])
print(y_test[0:10])

In [ ]:
# 검증되지 않은 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, 
                 y_pred=y_pred)
df_cm = pd.DataFrame(cm)
df_cm['sum']=df_cm.sum(axis=1)
df_cm.loc['Total']= df_cm.sum()
df_cm

그 혼돈 매트릭스는 이해하기 어려우니, 전에 만든 기능을 사용해서 더 예쁘게 만듬

In [ ]:
# Make a prettier confusion matrix

class_names=[str(i) for i in range(0,10)]
make_confusion_matrix(y_true=y_test, 
                      y_pred=y_pred,
                      classes=class_names,
                      figsize=(15, 15),
                      text_size=10)

# **Precision, Recall, and F1-score**
정밀도, 리콜 및 F1-점수

In [ ]:
# Let's consider the classification results for digit '1',

TP  = df_cm.iat[0,0]
FN = df_cm.iat[0, 10]-TP
FP = df_cm.iat[10, 0]-TP

precision = TP/(TP +FP)
recall = TP/(TP +FN)
F1 = 2*(precision*recall)/(precision+recall)

print("True Positive = {},  False Negative =  {},  False Positive = {}".format(TP,FN,FP))
print("\nPrecision = {:.4f},  Recall =  {:.4f},  F1 Score = {:.4f}".format(precision,recall,F1))

In [ ]:
# Use sklearn classification_report.

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, digits=4))

# **Inspecting the output**
출력 검사

출력을 검사하고 모든 것이 정상으로 보이는지 확인하는 것은 항상 좋은 생각입니다. 여기서는 정답과 정답이 틀린 예를 살펴보겠습니다.

In [ ]:
# predict_classes 함수는 가장 높은 확률 클래스를 출력합니다.
# 각 입력 예제에 대해 훈련된 분류자에 따라
# predicted_classes = model.predict_classes(X_test)

# 어떤 항목이 맞았는지/틀렸는지 확인
correct_indices = np.nonzero(predicted_classes == y_test)[0]

incorrect_indices = np.nonzero(predicted_classes != y_test)[0]


incorrect_indices.shape

# **Visualize FN**
FN 시각화

In [ ]:
# 앞서데이터 전처리과정에서에서 우리는 28 by 28 이차원 이미지데이터 1차원 벡터로 Reshape 시켜놓았다. 
# 이미지 플로팅을 위해 우리는 다시 이미지를 이차원으로 변화 시킨다.

x_test = X_test.reshape(10000, 28,28) 
\

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot FN classified images

def show_FN(x_test,y_test, y_pred, true_class):

  dict = {'True Class': y_test,'Predicted Class':y_pred}
  df = pd.DataFrame(dict)
  df['row_num'] = df.index
  df = df[df['True Class']==true_class]
  df = df[df['True Class'] != df['Predicted Class']]
  n = df.shape[0] # sample size
  print("Total Rows = ",n)
  print(df)

  fig = plt.figure(figsize=(10, 20))

  cols = 5
  rows = int(n/cols) +1

  for i in range(n):
    ax = plt.subplot(rows, cols, i+1)
    plt.imshow(x_test[df.iat[i,2]], cmap='gray')
    plt.title("True: {} Pred: {} ".format(df.iat[i,0],df.iat[i,1]))
    plt.axis(False)

show_FN(x_test,y_test, y_pred, 1)  # Check for 2, 8, 9

# **Visualize FP**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import math

# Plot FP classified images

def show_FP(x_test,y_test, y_pred, pred_class):

  dict = {'True Class': y_test,'Predicted Class':y_pred}
  df = pd.DataFrame(dict)
  df['row_num'] = df.index
  df = df[df['Predicted Class']==pred_class]
  df = df[df['True Class'] != df['Predicted Class']]
  n = df.shape[0] # sample size
  print("Total Rows = ",n)
  print(df)

  fig = plt.figure(figsize=(10, 20))

  cols = 5
  rows = int(n/cols) +1

  for i in range(n):
    ax = plt.subplot(rows, cols, i+1)
    plt.imshow(x_test[df.iat[i,2]], cmap='gray')
    plt.title("Pred {} True: {} ".format(df.iat[i,1],df.iat[i,0]))
    plt.axis(False)

In [ ]:
show_FP(x_test,y_test, y_pred, 3) # Check for cases predicted as 3 

In [ ]:
show_FP(x_test,y_test, y_pred, 7)